In [2]:
import itertools
import time
import os
import sys

In [3]:
import HTSeq
import pandas as pd

In [4]:
def invert_strand(iv):
    iv2 = iv.copy()
    if iv2.strand == "+":
        iv2.strand = "-"
    elif iv2.strand == "-":
        iv2.strand = "+"
    else:
        raise ValueError("Illegal strand")
    return iv2

In [5]:
# select gtf file: instantiate GFF_Reader Object
gtf_file = HTSeq.GFF_Reader("/gcm-lfs1/pablo/data/RNAdeg/annotation/gff/schizosaccharomyces_pombe.chr.extended.gff3")

In [6]:
# CIGAR match characters (including alignment match, sequence match, and
# sequence mismatch
com = ('M', '=', 'X')

In [7]:
## heterochromatic genes
htc_genes = ('dg1', 'dh1', 'after_tlh', 'MAT2', 'MAT3', 'MAT1')

# Counting ungapped single-end reads: **ChIP-Seq data**

- Obtain `GenomicArrayOfSets` for the `gene` **features** from the `GFF` **File** (for `DNA` **unstranded**)

In [10]:
# instantiate `GenomicArrayOfSets` for `gene` features: (`DNA` **unstranded**)
gene_features = HTSeq.GenomicArrayOfSets("auto", stranded=False)

genes_dict = {}
## for DNA we have to use the `gene` features 
#features_of_interest = ['gene', 'snRNA_gene', 'rRNA_gene', 'pseudogene', 'snoRNA_gene', 'tRNA_gene', 'ncRNA_gene']

## loop over all features in gtf file
for feature in gtf_file:
    
    ## parse features contained in `features_of_interest`
    #if feature.type in features_of_interest:
    if 'gene' in feature.type:
        
        # get `gene` feature id
        try:
            ## identify each `gene` feature by `gene_id` attribute: transcript/gene
            gene_id = feature.attr["gene_id"]

        except:
            ## sub-set of pseudogenes that behave as transcripts
            assert feature.type == 'pseudogene'
            gene_id = feature.attr["Parent"].split(':')[1]
        
        ## add `gene` feature to `GenomicArrayOfSets`
        gene_features[feature.iv] += gene_id

        # Is this the first time we see this gene?
        if gene_id not in genes_dict:
            # If so, add to the 'genes_dict' an empty list 
            genes_dict[ gene_id ] = list()

        # add the feature to the gene list
        genes_dict[ gene_id ].append( feature )

In [58]:
len(genes_dict)
#genes_dict

6992

![img_overlap](https://htseq.readthedocs.io/en/release_0.11.1/_images/count_modes.png)

- Parse `BAM` **File** to **count reads** falling into each `gene` **feature** 

In [12]:
#sample_name = '1022_INPUT'
sample_name = '1168_S2ChIP'

In [59]:
#os.path.join('/gcm-lfs1/pablo/data/RNAdeg/data/ChIP/bam', sample_name, sample_name + '.Aligned.sortedByCoord.out.bam')

In [104]:
import collections
counts = collections.Counter( )

start_time = time.time()

# select bam file: instantiate BAM_Reader Object (ChIP-seq)
#bam_file = HTSeq.BAM_Reader(os.path.join('/gcm-lfs1/pablo/data/RNAdeg/data/sequencing_new/ChIP/bam', sample_name, sample_name + '.Aligned.sortedByCoord.out.bam'))
#bam_file = HTSeq.BAM_Reader(os.path.join('/data/pablo/RNAdeg/results/ChIP/bams', sample_name + '.Aligned.sortedByCoord.out.bam'))
bam_file = HTSeq.BAM_Reader(os.path.join('/gcm-lfs1/pablo/data/RNAdeg/data/ChIP/bam', sample_name, sample_name + '.Aligned.sortedByCoord.out.bam'))

## -------
## Options
## -------
    
minaqual = 10

## How to deal with overlap of READ and FEATURE 
overlap_mode = "intersection-nonempty"
#overlap_mode =  "intersection-strict"

## How to deal with overlapping FEATURES and multimapped reads
multimapped_mode = 'all'
#multimapped_mode = 'none'

## How to deal with multimapped reads (secondary alignments!)
#secondary_alignment_mode = 'ignore'
secondary_alignment_mode = 'none'

i = 0

for aln in bam_file:
    
    if i > 0 and i % 100000 == 0:
        sys.stderr.write("{} alignment records processed. {} s\n".format(i,  time.time() - start_time))
        sys.stderr.flush()
    i += 1          
    
    ## ----------------------
    ## Inspect read alignment
    ## ----------------------
    
    ## _mapped or _unmapped (our BAM files only contain _mapped)
    counts["_total"] += 1
    
    if not aln.aligned:
        counts["_unmapped"] += 1
        ## skips to next iteration
        continue
    
    ## Multimapped reads are contained as separate entries in the BAM file.
    try:
        if aln.optional_field("NH") > 1:
            counts["_alignment_not_unique"] += 1
            if multimapped_mode == 'none':
                ## skips to next iteration
                continue
            elif ((secondary_alignment_mode == 'ignore') and aln.not_primary_alignment):
                counts["_not_primary_alignment"] += 1
                ## skips to next iteration
                continue

    except KeyError:
        pass

    #if aln.aQual < minaqual:
    #    #import pdb
    #    #pdb.set_trace()
    #    counts["_too_low_aQual"] +=  1
    #    continue

    ## -----------------------------
    ## Read and Feature Overlap Mode
    ## -----------------------------
    
    #iv_seq = (co.ref_iv for co in aln.cigar if (co.type in com and co.size > 0))
    
    ## A. Union: the union of all the sets S(i). This mode is recommended for most use cases.
    if overlap_mode == "union":
        ## feature set
        gene_ids = set()
        
        for iv, fs in gene_features[ aln.iv ].steps():
            gene_ids = gene_ids.union(fs)
    
    ## B. Intersection-strict: the intersection of all the sets S(i).
    ## C. Intersection-nonempty: the intersection of all non-empty sets S(i).
    elif overlap_mode in ("intersection-strict", "intersection-nonempty"):
        ## feature set
        gene_ids = None
        
        for iv, fs in gene_features[ aln.iv ].steps():
            if ((len(fs) > 0) or (overlap_mode == "intersection-strict")):
                if gene_ids is None:
                    gene_ids = fs.copy()
                else:
                    gene_ids = gene_ids.intersection(fs)
                    
    ## Other: Ilegal!                   
    else:
        sys.exit("Illegal overlap mode.")
    
    ## --------------
    ## Count Features
    ## --------------
    
    ## A. Mapped to unknown feature (it is empty)
    if gene_ids is None or len(gene_ids) == 0:
        counts["_no_feature"] += 1
        
    ## B. Mapped to a region with overlapping features (contains more than one element)
    ## See next how to deal with this ambiguous read alignments! (multimapped_mode)
    elif len(gene_ids) > 1:
        counts["_ambiguous"] += 1
    
    ## C. Uniquely Mapped (contains exactly one element) 
    #else:
    #    gene_id = list(gene_ids)[0]
    #    counts[gene_id] += 1
    
    ## Deal with multimapped reads!
    if gene_ids is not None and len(gene_ids) > 0:
        
        ## - ignore multimapped reads
        if multimapped_mode == 'none':
            
            ## C. Uniquely Mapped (contains exactly one element) 
            if len(gene_ids) == 1:
                counts[list(gene_ids)[0]] += 1
        
        ## - count each multimapped feature
        elif multimapped_mode == 'all':
            for fsi in list(gene_ids):
                counts[fsi] += 1
        
        ## Other: Ilegal!                   
        else:
            sys.exit("Illegal multimap mode.")

        
print('Elapsed Time (Counting reads):', time.time() - start_time)

100000 alignment records processed. 4.7998435497283936 s
200000 alignment records processed. 9.677944421768188 s
300000 alignment records processed. 14.543773174285889 s
400000 alignment records processed. 19.359629154205322 s
500000 alignment records processed. 24.30399513244629 s
600000 alignment records processed. 29.221397161483765 s
700000 alignment records processed. 34.09536600112915 s
800000 alignment records processed. 38.94419836997986 s
900000 alignment records processed. 43.72959589958191 s
1000000 alignment records processed. 48.63942074775696 s
1100000 alignment records processed. 53.55071687698364 s
1200000 alignment records processed. 58.414520502090454 s
1300000 alignment records processed. 63.20249271392822 s
1400000 alignment records processed. 68.05722284317017 s
1500000 alignment records processed. 72.85994577407837 s
1600000 alignment records processed. 77.65871262550354 s
1700000 alignment records processed. 82.51195955276489 s
1800000 alignment records processed

Elapsed Time (Counting reads): 134.8384654521942


In [89]:
#for gene_id in counts:
#    print(gene_id, counts[gene_id])

In [90]:
#{k: v for k, v in sorted(counts.items(), key=lambda item: item[1], reverse=True)}
#{k: v for k, v in counts.items()}

- Convert `counter` to **DataFrame**

In [105]:
counts_df = pd.DataFrame.from_dict(counts, orient='index').reset_index()
counts_df = counts_df.rename(columns={'index':'gene-id', 0:'count'})
counts_df.head(20)

,gene-id,count
0,_total,2785741
1,_alignment_not_unique,304271
2,SPAC212.11,1074
3,SPAC212.10,137
4,_no_feature,343544
5,SPAC212.09c,372
6,SPNCRNA.70,32
7,SPAC212.08c,145
8,SPAC212.07c,50
9,SPAC212.12,80


In [106]:
counts_df.shape

(6845, 2)

- Summary of counts

In [107]:
counts_df[counts_df['gene-id'].str.startswith('_')]

,gene-id,count
0,_total,2785741
1,_alignment_not_unique,304271
4,_no_feature,343544
17,_ambiguous,344488


In [108]:
## now contains multiple counting
counts_df[~counts_df['gene-id'].str.startswith('_')]['count'].sum()

2803658

- Show counts for genes of interest: **Heterochromatic Genes**

In [109]:
counts_df[counts_df['gene-id'].isin(htc_genes)]

,gene-id,count
2072,dh1,1345
2076,dg1,1198
4212,MAT1,24
4224,MAT2,172
4226,MAT3,224
5545,after_tlh,622


## Compare results with **Parastou's counting**

- Results using **Parastou's counting script**

In [96]:
xp_chip = '/gcm-lfs1/pablo/data/RNAdeg/data/ChIP/xp_data/chip_pombe_gene_count_matrix.csv'

In [97]:
xp_chip_df = pd.read_csv(xp_chip, sep='\t')
xp_chip_df = xp_chip_df[['gene-id', 'length', 'type', 'category', 'bio_type', sample_name]].astype({sample_name: 'int64'})
xp_chip_df.head()

,gene-id,length,type,category,bio_type,1168_S2ChIP
0,SPAC212.11,5661.0,gene,repeat,protein_coding,1074
1,SPAC212.10,605.0,pseudogene,repeat,pseudogene,137
2,SPAC212.09c,1655.0,pseudogene,repeat,pseudogene,372
3,SPNCRNA.70,529.0,ncRNA_gene,repeat,ncRNA,32
4,SPAC212.08c,1210.0,gene,repeat,protein_coding,145


In [98]:
total = xp_chip_df[sample_name].sum()
total

2639350

- Show counts for genes of interest: **Heterochromatic Genes**

In [99]:
xp_chip_df[xp_chip_df['gene-id'].isin(htc_genes)]

,gene-id,length,type,category,bio_type,1168_S2ChIP
6937,dg1,3500.0,gene,repeat,protein_coding,1198
6938,dh1,4000.0,gene,repeat,protein_coding,1345
6939,after_tlh,5000.0,gene,repeat,protein_coding,540
6940,MAT2,4000.0,gene,repeat,protein_coding,172
6941,MAT3,4000.0,gene,repeat,protein_coding,238
6942,MAT1,1500.0,gene,repeat,protein_coding,24


- **Merge** both counts DataFrames

In [100]:
merged_xp_chip = pd.merge(counts_df, xp_chip_df, on='gene-id', how='outer')
merged_xp_chip

,gene-id,count,length,type,category,bio_type,1168_S2ChIP
0,_total,2785741.0,NaN,NaN,NaN,NaN,NaN
1,_alignment_not_unique,304271.0,NaN,NaN,NaN,NaN,NaN
2,SPAC212.11,1064.0,5661.0,gene,repeat,protein_coding,1074.0
3,_no_feature,447609.0,NaN,NaN,NaN,NaN,NaN
4,SPAC212.10,124.0,605.0,pseudogene,repeat,pseudogene,137.0
...,...,...,...,...,...,...,...
6978,SPMITTRNAVAL.01,NaN,72.0,tRNA_gene,gene,tRNA,33.0
6979,SPMITTRNATYR.01,NaN,85.0,tRNA_gene,gene,tRNA,2.0
6980,SPMITTRNASER.02,NaN,81.0,tRNA_gene,gene,tRNA,36.0
6981,SPMITTRNAALA.01,NaN,72.0,tRNA_gene,gene,tRNA,9.0


In [101]:
merged_xp_chip[merged_xp_chip['gene-id'].isin(htc_genes)]

,gene-id,count,length,type,category,bio_type,1168_S2ChIP
2046,dh1,1335.0,4000.0,gene,repeat,protein_coding,1345.0
2050,dg1,1198.0,3500.0,gene,repeat,protein_coding,1198.0
4159,MAT1,24.0,1500.0,gene,repeat,protein_coding,24.0
4171,MAT2,155.0,4000.0,gene,repeat,protein_coding,172.0
4173,MAT3,224.0,4000.0,gene,repeat,protein_coding,238.0
5487,after_tlh,622.0,5000.0,gene,repeat,protein_coding,540.0


In [38]:
select_genes = ['SPCC569.04', 'SPCC569.05c']

In [39]:
merged_xp_chip[merged_xp_chip['gene-id'].isin(select_genes)].reset_index(drop=True)
#merged_xp_chip[merged_xp_chip['gene-id'] == 'SPCC569.05c'].reset_index(drop=True)

,gene-id,count,length,type,category,bio_type,1168_S2ChIP
0,SPCC569.05c,346.0,4188.0,gene,gene,protein_coding,356.0
1,SPCC569.04,12.0,544.0,gene,gene,protein_coding,18.0


In [41]:
iv_test = HTSeq.GenomicInterval("MT", 11190, 12500, ".")

gene_ids = set()
## loop over features overlapping with the read
for iv, val in gene_features[ iv_test ].steps():
    gene_ids |= val

gene_ids

{'SPMIT.05', 'SPMIT.06'}

----

## Counting gapped single-end reads:  **RNA-Seq data**

The above code can be used as is e.g. for **ChIP-Seq data**, but for **RNA-Seq data**, we need an additional ingredient.

When sequencing RNA, many reads will pass over an **exon-exon junction** and hence **align to two (or more) disjunct intervals** on the genome, 
tyically with an intron in between.
If the reads have been aligned with a **splice-aware alignment tool** (e.g. STAR), such gapped alignment is indicated in the `SAM` **file** by the `CIGAR` (Compact Idiosyncratic Gapped Alignment Report) **string**.

- Obtain `GenomicArrayOfSets` for the `exon` **features** from the `GFF` **File** (for `RNA` **stranded**)

In [8]:
# instantiate `GenomicArrayOfSets` for the `exons` features:
exons = HTSeq.GenomicArrayOfSets("auto", stranded=True)
#exons = HTSeq.GenomicArrayOfSets("auto", stranded=False)

## loop over all features in gtf file
for feature in gtf_file:
    
    ## store all exons in our `GenomicArrayOfSets`
    if feature.type == "exon":
        
        ## identify each `exon` feature by parent transcript/gene
        #exons[feature.iv] += feature.attr["Name"].split(':')[0]
        #exons[feature.iv] += feature.attr["Parent"].split(':')[1]
        exons[feature.iv] += feature.attr["Parent"].split(':')[1][:-2] ## get rid of '.1'


In [9]:
aln.cigar

NameError: name 'aln' is not defined

In [10]:
#aln.cigar[1].__dir__()

In [90]:
aln.cigar[1].type

'M'

In [93]:
invert_strand(aln.cigar[1].ref_iv)

<GenomicInterval object 'I', [5670,5703), strand '+'>

In [58]:
#iv_seq = (invert_strand(co.ref_iv) for co in aln.cigar if (co.type in com and co.size > 0))
iv_seq = (co.ref_iv for co in aln.cigar if (co.type in com and co.size > 0))

In [59]:
for iv in iv_seq:
#for cig_op in aln.cigar:
    
    print(iv)
    ## different from 'matched'
    #if cig_op.type != "M":
        ## skips to next iteration
        #continue

    ## loop over features overlapping with the cig_op
    #for iv, val in exons[ cig_op.ref_iv ].steps():
    for iv2, val2 in exons[ iv ].steps():
        gene_ids |= val2


I:[459,509)/-


In [60]:
gene_ids

{'SPAC212.11.1'}

In [12]:
import collections
counts = collections.Counter( )

start_time = time.time()

# select bam file: instantiate BAM_Reader Object (RNA-seq)
bam_file = HTSeq.BAM_Reader('/gcm-lfs1/pablo/data/RNAdeg/data/RNA/bams/1113_pA/1113_pA.Aligned.sortedByCoord.out.bam')

for aln in itertools.islice(bam_file, 10000):
#for aln in bam_file:
    
    ## _mapped or _unmapped
    counts["_total"] += 1
    
    if not aln.aligned:
        counts["_unmapped"] += 1
        ## skips to next iteration
        continue
        
    gene_ids = set()
    
    ## invert strand - due to sequencing is strand is reversed!
    iv_seq = (invert_strand(co.ref_iv) for co in aln.cigar if (co.type in com and co.size > 0))
    #iv_seq = (co.ref_iv for co in aln.cigar if (co.type in com and co.size > 0))
    
    ## loop over CIGAR operations (cig_op) for aligned read    
    for iv in iv_seq:
    #for cig_op in aln.cigar:

        ## different from 'matched'
        #if cig_op.type != "M":
            ## skips to next iteration
            #continue
        
        ## loop over features overlapping with the cig_op
        #for iv, val in exons[ cig_op.ref_iv ].steps():
        for iv2, val2 in exons[ iv ].steps():
            gene_ids |= val2
    
    ## Count Genes:
    if len(gene_ids) == 1:
        ## A. Uniquely Mapped (contains exactly one element) 
        gene_id = list(gene_ids)[0]
        counts[gene_id] += 1
        
    elif len(gene_ids) == 0:
        #import pdb
        #pdb.set_trace()
        ## B. Mapped to unknown feature (it is empty)
        counts["_no_feature"] += 1
        
    else:
        #import pdb
        #pdb.set_trace()
        ## C. Mapped to overlapping feature (contains more than one element)
        counts["_ambiguous"] += 1
        
print('Elapsed Time (Counting reads):', time.time() - start_time)

Elapsed Time (Counting reads): 0.4896712303161621


In [73]:
{k: v for k, v in sorted(counts.items(), key=lambda item: item[1], reverse=True)}

{'_total': 10000,
 'SPAC212.01c.1': 2716,
 'SPAC212.04c.1': 2444,
 'SPAC977.01.1': 1197,
 'SPAC212.09c.1': 870,
 '_no_feature': 809,
 'SPAC212.11.1': 552,
 'SPAC212.08c.1': 381,
 'SPAC212.12.1': 279,
 'SPAC212.06c.1': 233,
 'SPAC212.03.1': 153,
 'SPAC212.02.1': 70,
 '_ambiguous': 67,
 'SPAC212.07c.1': 60,
 'SPAC13G6.07c.1': 30,
 'SPAC977.14c.1': 24,
 'SPAC5H10.13c.1': 15,
 'SPAC11D3.16c.1': 13,
 'SPAC1F8.07c.1': 13,
 'SPAC24B11.13.1': 11,
 'SPNCRNA.70.1': 10,
 'SPAC212.10.1': 9,
 'SPAPJ695.02.1': 7,
 'SPAC24B11.06c.1': 7,
 'SPAC212.05c.1': 7,
 'SPAC977.11.1': 6,
 'SPAC13G6.02c.1': 6,
 'SPAC13G6.10c.1': 2,
 'SPAC11D3.17.1': 2,
 'SPNCRNA.648.1': 1,
 'SPAC24B11.09.1': 1,
 'SPAC11D3.11c.1': 1,
 'SPAC5H10.03.1': 1,
 'SPAC13G6.01c.1': 1,
 'SPAC977.12.1': 1,
 'SPAC1639.01c.1': 1}

## Counting gapped single-end reads:  **HTSeq-count**

![img_overlap](https://htseq.readthedocs.io/en/release_0.11.1/_images/count_modes.png)

In [1]:
import collections
counts = collections.Counter( )

start_time = time.time()

# select bam file: instantiate BAM_Reader Object (ChIP-seq)
#bam_file = HTSeq.BAM_Reader(os.path.join('/gcm-lfs1/pablo/data/RNAdeg/data/sequencing_new/ChIP/bam', sample_name, sample_name + '.Aligned.sortedByCoord.out.bam'))
#bam_file = HTSeq.BAM_Reader(os.path.join('/data/pablo/RNAdeg/results/ChIP/bams', sample_name + '.Aligned.sortedByCoord.out.bam'))
bam_file = HTSeq.BAM_Reader(os.path.join('/gcm-lfs1/pablo/data/RNAdeg/data/ChIP/bam', sample_name, sample_name + '.Aligned.sortedByCoord.out.bam'))

overlap_mode = "intersection-nonempty"

for aln in bam_file:
    
    ## _mapped or _unmapped
    counts["_total"] += 1
    
    if not aln.aligned:
        counts["_unmapped"] += 1
        ## skips to next iteration
        continue
    
    ## ------------
    ## Overlap Mode
    ## ------------
    
    ## invert strand - due to sequencing is strand is reversed!
    iv_seq = (invert_strand(co.ref_iv) for co in aln.cigar if (co.type in com and co.size > 0))
    #iv_seq = (co.ref_iv for co in aln.cigar if (co.type in com and co.size > 0))
    
    ## A. Union: the union of all the sets S(i). This mode is recommended for most use cases.
    if overlap_mode == "union":
        ## feature set
        gene_ids = set()
        for iv in iv_seq:
            #if iv.chrom not in features.chrom_vectors:
            #    raise UnknownChrom
            for iv2, fs2 in gene_features[ iv ].steps():
                gene_ids = gene_ids.union(fs2)
    
    ## B. Intersection-strict: the intersection of all the sets S(i).
    ## C. Intersection-nonempty: the intersection of all non-empty sets S(i).
    elif overlap_mode in ("intersection-strict", "intersection-nonempty"):
        ## feature set
        gene_ids = None
        for iv in iv_seq:
            #if iv.chrom not in features.chrom_vectors:
            #    raise UnknownChrom
            for iv2, fs2 in gene_features[ iv ].steps():
                if ((len(fs2) > 0) or (overlap_mode == "intersection-strict")):
                    if gene_ids is None:
                        gene_ids = fs2.copy()
                    else:
                        gene_ids = gene_ids.intersection(fs2)
    ## Other: Ilegal!                   
    else:
        sys.exit("Illegal overlap mode.")
    
    ## --------------
    ## Count Features
    ## --------------

    if gene_ids is None or len(gene_ids) == 0:
        ## A. Mapped to unknown feature (it is empty)
        counts["_no_feature"] += 1
        import pdb
        pdb.set_trace()
        
    elif len(gene_ids) > 1:
        ## B. Mapped to overlapping feature (contains more than one element)
        counts["_ambiguous"] += 1
    
    else:
        ## C. Uniquely Mapped (contains exactly one element) 
        gene_id = list(gene_ids)[0]
        counts[gene_id] += 1
        
    ## Deal with multimapped reads! fs: feature set
    #if fs is not None and len(fs) > 0:
    #    
    #    if multimapped_mode == 'none':
    #        if len(fs) == 1:
    #            counts[list(fs)[0]] += 1
    #            
    #    elif multimapped_mode == 'all':
    #        for fsi in list(fs):
    #            counts[fsi] += 1
    #    
    #    ## Other: Ilegal!                   
    #    else:
    #        sys.exit("Illegal multimap mode.")

        
print('Elapsed Time (Counting reads):', time.time() - start_time)

NameError: name 'time' is not defined

In [ ]:
try:
    ## ------------
    ## Overlap Mode
    ## ------------

    ## A. Union: the union of all the sets S(i). This mode is recommended for most use cases.
    if overlap_mode == "union":
        ## feature set
        fs = set()
        for iv in iv_seq:
            #if iv.chrom not in features.chrom_vectors:
            #    raise UnknownChrom
            for iv2, fs2 in features[ iv ].steps():
                fs = fs.union(fs2)
    
    ## B. Intersection-strict: the intersection of all the sets S(i).
    ## C. Intersection-nonempty: the intersection of all non-empty sets S(i).
    elif overlap_mode in ("intersection-strict", "intersection-nonempty"):
        ## feature set
        fs = None
        for iv in iv_seq:
            #if iv.chrom not in features.chrom_vectors:
            #    raise UnknownChrom
            for iv2, fs2 in features[iv].steps():
                if ((len(fs2) > 0) or (overlap_mode == "intersection-strict")):
                    if fs is None:
                        fs = fs2.copy()
                    else:
                        fs = fs.intersection(fs2)
    ## Other: Ilegal!                   
    else:
        sys.exit("Illegal overlap mode.")
        
    ## --------------
    ## Count Features
    ## --------------

    if fs is None or len(fs) == 0:
        ## A. Mapped to unknown feature (it is empty)
        write_to_samout(r, "__no_feature", samoutfile)
        empty += 1
        
    elif len(fs) > 1:
        ## B. Mapped to overlapping feature (contains more than one element)
        write_to_samout(r, "__ambiguous[" + '+'.join(fs) + "]", samoutfile)
        ambiguous += 1
    
    else:
        ## C. Uniquely Mapped (contains exactly one element) 
        write_to_samout(r, list(fs)[0], samoutfile)
    
    ## Deal with multimapped reads!
    if fs is not None and len(fs) > 0:
        
        if multimapped_mode == 'none':
            if len(fs) == 1:
                counts[list(fs)[0]] += 1
                
        elif multimapped_mode == 'all':
            for fsi in list(fs):
                counts[fsi] += 1
        
        ## Other: Ilegal!                   
        else:
            sys.exit("Illegal multimap mode.")


## Counting gapped single-end reads:  **Parastou's script**